<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/02-Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages and Setup Variables

In [20]:
!pip install -q openai==1.6.0 cohere==4.39 tiktoken==0.5.2 pandas==2.2.0 scikit-learn==1.4.1.post1

In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

In [3]:
# False: Generate the embedding for the dataset. (Associated cost with using OpenAI endpoint)
# True: Load the dataset that already has the embedding vectors.
load_embedding = False

# Load Dataset

## Download Dataset (JSON)

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.

In [9]:
!curl -o ./mini-dataset.json https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-dataset.json if running locally on bash
!curl -o ./mini-dataset_with_embedding.json https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-dataset_with_embedding.json if running locally on bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25361  100 25361    0     0   346k      0 --:--:-- --:--:-- --:--:--  348k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  465k  100  465k    0     0  3732k      0 --:--:-- --:--:-- --:--:--     0--:--:-- --:--:-- 3755k


## Read File

In [10]:
import json

# Load the file as a JSON
with open('./mini-dataset.json', 'r') as file:
    data = json.load(file)

# The number of chunks in the essay dataset.
len( data['chunks'] )

22

In [13]:
import pandas as pd

# Convert the JSON list to a Pandas Dataframe
df = pd.DataFrame(data['chunks'])

df.keys()

/var/folders/l7/9qcp7g5x5rl9x8ltw0t85qym0000gn/T/ipykernel_99137/3242328735.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Index(['text', 'embedding'], dtype='object')

# Generate Embedding

In [14]:
from openai import OpenAI

client = OpenAI()

# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
  try:
    # Remove newlines
    text = text.replace("\n", " ")
    res = client.embeddings.create(input = [text], model="text-embedding-ada-002")

    return res.data[0].embedding

  except:
        return None

In [16]:
from tqdm import tqdm

# Generate embedding
if not load_embedding:
  print("Generating embeddings...")
  for index, row in tqdm( df.iterrows() ):
    row['embedding'] = get_embedding( row['text'] )

# Or, load the embedding from the file.
else:
  print("Loaded the embedding file.")
  with open('./mini-dataset_with_embedding.json', 'r') as file:
    data = json.load(file)
  df = pd.DataFrame(data)

Generating embeddings...


22it [00:04,  4.99it/s]


# User Question

In [17]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding( QUESTION )

len( QUESTION_emb )

1536

# Test Cosine Similarity

Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.

In [18]:
BAD_SOURCE_emb = get_embedding( "The sky is blue." )
GOOD_SOURCE_emb = get_embedding( "LLaMA2 model has a total of 2B parameters." )

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity([QUESTION_emb], [BAD_SOURCE_emb]) )
print("> Good Response Score:", cosine_similarity([QUESTION_emb], [GOOD_SOURCE_emb]) )

> Bad Response Score: [[0.69954666]]
> Good Response Score: [[0.93135959]]


# Calculate Cosine Similarities

In [22]:
# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity( [QUESTION_emb], df['embedding'].tolist() )

print( cosine_similarities )

[[0.81723369 0.86596983 0.80878332 0.76617099 0.83996096 0.79028881
  0.78438889 0.80187419 0.82465877 0.81997573 0.84173789 0.75347142
  0.82343033 0.70476467 0.656007   0.73980692 0.76207973 0.76291356
  0.79877322 0.73906994 0.78277211 0.75805319]]


In [23]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax( cosine_similarities )

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print( indices )

[ 1 10  4]


In [24]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate( df.text[indices] ):
  print(f"> Chunk {idx+1}")
  print(item)
  print("----")

> Chunk 1
II. Llama 2 Model Flavors: Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited. The pretrained variant, trained on a whopping 2 trillion tokens, boasts a context window of 4096 tokens, twice the size of its predecessor Llama 1. Meta also released a Llama 2 fine-tuned model for chat applications that was trained on over 1 million human annotations. Such extensive training comes at a cost, with the 70B model taking a staggering 1720320 GPU hours to train. The context window’s length determines the amount of content the model can process at once, making Llama 2 a powerful language model in terms of scale and efficiency.
----
> Chunk 2
New Llama-2 model: In mid-July, Meta released its new family of pre-trained and finetuned models called Llama-2, with an open source and commercial character to facilitate its use and expansion. The base mo

# Augment the Prompt

In [25]:
# Use the OpenAI API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = (
      "You are an assistant and expert in answering questions from a chunks of content. "
      "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
      "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
      "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format( "".join( df.text[indices] ), QUESTION )

    # Call the OpenAI API
    response = client.chat.completions.create(
      model='gpt-3.5-turbo-0125',
      temperature=0.0,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
      ]
    )

    # Return the AI's response
    res = response.choices[0].message.content.strip()

except Exception as e:
    print( f"An error occurred: {e}" )

In [26]:
print( res )

The Llama 2 model has four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.


# Without Augmentation

Test the OpenAI API to answer the same question without the addition of retrieved documents. Basically, the LLM will use its knowledge to answer the question.

In [33]:
# Formulating the system prompt
system_prompt = (
  "You are an assistant and expert in answering questions."
)

# Combining the system prompt with the user's question
prompt = (
  "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
)
prompt = prompt.format( QUESTION )

# Call the OpenAI API
response = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  temperature=.9,
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
  ]
)

# Return the AI's response
res = response.choices[0].message.content.strip()

In [34]:
print( res )

The LLaMA2 model has a total of X parameters.
